Poisson Equation With Neumann Boundary Conditions
===

Import Netgen/NGSolve Python modules:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ngsolve import *
from netgen.geom2d import unit_square
import netgen.gui
%gui tk

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2305
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 8 thread(s)


# Problem

\omega = (0,1)^2

In [2]:
ud = CoefficientFunction(x)
g = CoefficientFunction(y)
alpha = 5

order = 2
size = 0.1
mesh = Mesh(unit_square.GenerateMesh(maxh=size))


 Generate Mesh from spline geometry


In [3]:
#Compare with H1
H = H1(mesh, order=order, dirichlet="right|top|bottom")
u,v = H.TnT()

A = BilinearForm(H, symmetric=True)
A += SymbolicBFI( grad(u)*grad(v) - (1/alpha)*u*v )
A.Assemble()

f = LinearForm(H)
f += SymbolicLFI( 1*v )
f.Assemble()

gfu = GridFunction(H)
gfu.vec.data = A.mat.Inverse(H.FreeDofs())*f.vec
Draw(gfu, mesh, "sigma")



In [4]:
V = HDiv(mesh, order=order, dirichlet="left")
Q = L2(mesh, order=order-1)
X = FESpace( [V,Q] )
(sigma, u), (tau, v) = X.TnT()

A = BilinearForm(X, symmetric=True)
A += SymbolicBFI( sigma*tau + div(tau)*u + div(sigma)*v - 1/alpha*u*v )
A.Assemble()

normal = specialcf.normal(mesh.dim)
f = LinearForm(X)
f += SymbolicLFI( -1*v )
f += SymbolicLFI(ud * (tau.Trace()*normal), BND)
f.Assemble()

gfsol = GridFunction(X)
gfsol.vec.data = A.mat.Inverse(X.FreeDofs())*f.vec

Draw(gfsol.components[0], mesh, "sigma")
Draw(gfsol.components[1], mesh, "u")




In [5]:
print("diff u    = ", sqrt(Integrate((gfsol.components[1]-gfu)**2, mesh)) )
print("diff flux = ", sqrt(Integrate((gfsol.components[0]-grad(gfu))**2, mesh)) )


diff u    =  0.6133269405692692
diff flux =  0.8546858773855345
